
(himmelblau)=
## Himmelblau's function

Himmelblau's function is another famous mathematical function used as a benchmark problem in optimization. It's named after David Mautner Himmelblau, who introduced it in 1972. The function is defined as follows:

```{figure} ./../img/%202024-04-30-14-55-45.png.png
---
align: center
---

Himmelblau's function
Source: https://commons.wikimedia.org/wiki/File:Himmelblau_function.svg.
Image by Morn the Gorn. Released to the public domain

```

Although this function seems simpler in comparison to the Eggholder function, it draws
interest as it is multi-modal, in other words, it has more than one global minimum. To be
exact, the function has four global minima evaluating to 0, which can be found in the
following locations:

````
x=3.0, y=2.0
x=−2.805118, y=3.131312
x=−3.779310, y=−3.283186
x=3.584458, y=−1.848126
```

Which can be despicted here:

```{figure} ./../img/2024-04-30-15-25-27.png
---
align: center
---

Contour diagram of Himmelblau's function
Source: https://commons.wikimedia.org/wiki/File:Himmelblau_contour.svg. Image by: Nicoguaro.
Licensed under Creative Commons CC BY 4.0: https://creativecommons.org/licenses/by/4.0/deed.en.

```



```{math}
f(x,y) = (x^2 + y - 11)^2 + (x + y^2 - 7)^2 
```
Let's break it down:

1. **The Structure**: Himmelblau's function is made up of two polynomial terms, each squared and added together.

2. **The Variables**: The function depends on two variables, \( x \) and \( y \), which represent coordinates on a 2D plane.

3. **The Squares**: Each term in the function is squared. Squaring ensures that all parts of the function are positive, which can create multiple local minima and maxima.

4. **The Constants**: The numbers 11 and 7 in the function are constants.

5. **The Summation**: The squared terms are added together to form a single value, which represents the height of the function's surface at a given point \( (x,y) \).

6. **The Objective**: The goal is to find the minimum value of the function, which corresponds to the lowest point on its surface.

Himmelblau's function is particularly interesting because it has four identical local minima, each with a value of 0, and one global minimum also at 0. These minima are located at the points:

```{math}
(3, 2), \ (-2.805118, 3.131312), \ (-3.779310, -3.283186), \ \text{and} \ (3.584428, -1.848126)
```

These points are like valleys in the landscape represented by the function, and finding them can be challenging due to the function's complex structure with multiple hills and valleys.

In optimization, scientists and engineers use Himmelblau's function to test and compare different optimization algorithms. The efficiency of an algorithm is measured by how quickly and accurately it can find the global minimum (the lowest point) of the function. If an algorithm can efficiently navigate the landscape of Himmelblau's function to find the minima, it's likely to perform well on other optimization problems too.


In [ ]:
from deap import base
from deap import creator
from deap import tools

import random
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import elitism

def run_himmelblau_optimization_genetic(DIMENSIONS = 2, BOUND_LOW = -5.0, BOUND_UP = 5.0, POPULATION_SIZE = 300, P_CROSSOVER = 0.9, P_MUTATION = 0.5, MAX_GENERATIONS = 300, HALL_OF_FAME_SIZE = 30, CROWDING_FACTOR = 20.0, RANDOM_SEED = 42):

    random.seed(RANDOM_SEED)

    toolbox = base.Toolbox()

    # define a single objective, minimizing fitness strategy:
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

    # create the Individual class based on list:
    creator.create("Individual", list, fitness=creator.FitnessMin)


    # helper function for creating random float numbers uniformaly distributed within a given range [low, up]
    # it assumes that the range is the same for every dimension
    def randomFloat(low, up):
        return [random.uniform(a, b) for a, b in zip([low] * DIMENSIONS, [up] * DIMENSIONS)]

    # create an operator that randomly returns a float in the desired range and dimension:
    toolbox.register("attr_float", randomFloat, BOUND_LOW, BOUND_UP)

    # create the individual operator to fill up an Individual instance:
    toolbox.register("individualCreator", tools.initIterate, creator.Individual, toolbox.attr_float)

    # create the population operator to generate a list of individuals:
    toolbox.register("populationCreator", tools.initRepeat, list, toolbox.individualCreator)


    # Himmelblau function as the given individual's fitness:
    def himmelblau(individual):
        x = individual[0]
        y = individual[1]
        f = (x ** 2 + y - 11) ** 2 + (x + y ** 2 - 7) ** 2
        return f,  # return a tuple

    toolbox.register("evaluate", himmelblau)

    # genetic operators:
    toolbox.register("select", tools.selTournament, tournsize=2)
    toolbox.register("mate", tools.cxSimulatedBinaryBounded, low=BOUND_LOW, up=BOUND_UP, eta=CROWDING_FACTOR)
    toolbox.register("mutate", tools.mutPolynomialBounded, low=BOUND_LOW, up=BOUND_UP, eta=CROWDING_FACTOR, indpb=1.0/DIMENSIONS)


    # create initial population (generation 0):
    population = toolbox.populationCreator(n=POPULATION_SIZE)

    # prepare the statistics object:
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("min", np.min)
    stats.register("avg", np.mean)

    # define the hall-of-fame object:
    hof = tools.HallOfFame(HALL_OF_FAME_SIZE)

    # perform the Genetic Algorithm flow with elitism:
    population, logbook = elitism.eaSimpleWithElitism(population, toolbox, cxpb=P_CROSSOVER, mutpb=P_MUTATION,
                                                ngen=MAX_GENERATIONS, stats=stats, halloffame=hof, verbose=True)

    # print info for best solution found:
    best = hof.items[0]
    print("-- Best Individual = ", best)
    print("-- Best Fitness = ", best.fitness.values[0])

    print("- Best solutions are:")
    for i in range(HALL_OF_FAME_SIZE):
        print(i, ": ", hof.items[i].fitness.values[0], " -> ", hof.items[i])


    # extract statistics:
    minFitnessValues, meanFitnessValues = logbook.select("min", "avg")
    return minFitnessValues, meanFitnessValues, population


minFitnessValues_42, meanFitnessValues_42, population_42 = run_himmelblau_optimization_genetic(RANDOM_SEED=42)

# random with seed 13 for comparison:
minFitnessValues_13, meanFitnessValues_13, population_13 = run_himmelblau_optimization_genetic(RANDOM_SEED=13)

# plot statistics:


# plot solution locations on x-y plane:
plt.figure(1)
globalMinima = [[3.0, 2.0], [-2.805118, 3.131312], [-3.779310, -3.283186], [3.584458, -1.848126]]
plt.scatter(*zip(*globalMinima), marker='X', color='red', zorder=1)
plt.scatter(*zip(*population_42), marker='.', color='blue', zorder=0)
plt.scatter(*zip(*population_13), marker='.', color='green', zorder=0)

# Legends and labels.
plt.legend(['Global Minima', '42', '13'])


plt.figure(2)
sns.set_style("whitegrid")
plt.plot(minFitnessValues_42, color='red')
plt.plot(meanFitnessValues_42, color='green')
plt.plot(minFitnessValues_13, color='blue')

plt.xlabel('Generation')
plt.ylabel('Min / Average Fitness')
plt.title('Min and Average fitness over Generations')

# Legends.
plt.legend(['42', '42 Mean', '13'])

plt.show()


